## Test ImageNet models preprocessing

### First load in the model

In [3]:
# Imports
import numpy as np
   
from keras.optimizers import SGD
from densenet161 import DenseNet 
from load_data_imagenet import load_data_imagenet_split
from resnet152 import resnet152_model
import keras
from calibration import evaluate_model
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
import time


Using TensorFlow backend.


In [7]:
# Constants (UPPERCASE)
seed = 333
num_classes = 1000
weights_file_resnet = "../../models/resnet152_weights_tf.h5"
weights_file_densenet = "../../models/densenet161_weights_tf.h5"
MEAN = [103.939, 116.779, 123.68]


In [5]:
# Load model
time1 = time.time()
model = resnet152_model()
sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
# Load weights
model.load_weights(weights_file_resnet, by_name=True)
time2 = time.time()
print('It took %0.3f ms' % ((time2-time1)*1000.0))

It took 39215.237 ms


### Next load in image

In [8]:
time1 = time.time()

img_path = "test_img/9.JPEG"
img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

x = x[..., ::-1]

for i in range(3):
    x[:,:,:,i] -= MEAN[i]

print('Input image shape:', x.shape)


preds = model.predict(x)
print(np.argmax(preds, axis=1))
print('Predicted:', decode_predictions(preds))
time2 = time.time()
print('It took %0.3f ms' % ((time2-time1)*1000.0))

Input image shape: (1, 224, 224, 3)
[982]
Predicted: [[('n10148035', 'groom', 0.99997485), ('n03450230', 'gown', 2.089943e-05), ('n02883205', 'bow_tie', 1.9427303e-06), ('n03476991', 'hair_spray', 1.4953844e-06), ('n04350905', 'suit', 2.9126187e-07)]]
It took 3044.039 ms


In [32]:
### ImageNet loading like in Evaluation script

from load_data_imagenet import load_img, center_crop

time1 = time.time()

img_path = "test_img/5.JPEG"
img = load_img(img_path)
img = center_crop(img)

x = np.expand_dims(img, axis=0)
x.shape
x = np.array(x, dtype="float32")
print(x.dtype)

x = x[..., ::-1]

for i in range(3):
    x[:,:,:,i] -= MEAN[i]
    
preds = model.predict(x)
print(np.argmax(preds, axis=1))
print('Predicted:', decode_predictions(preds))
time2 = time.time()
print('It took %0.3f ms' % ((time2-time1)*1000.0))

float32
[644]
Predicted: [[('n03729826', 'matchstick', 0.99998772), ('n03134739', 'croquet_ball', 6.9314724e-06), ('n03970156', 'plunger', 1.2312286e-06), ('n03976657', 'pole', 1.0859825e-06), ('n03250847', 'drumstick', 6.4395869e-07)]]
It took 2036.415 ms


### Same for DenseNet

In [158]:
im = image.img_to_array(img)
im = np.expand_dims(im, axis=0)
im.shape

(1, 224, 224, 3)

In [159]:
im.ndim

4

In [160]:
print(im[:,1,1,:])

[[ 233.   25.   51.]]


(1, 224, 224, 3)
[[  51.   25.  233.]]
[[ -52.93900299  -91.77899933  109.31999969]]


In [51]:
time1 = time.time()
model2 = DenseNet(reduction=0.5, classes=1000)
sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
model2.load_weights(weights_file_densenet, by_name=True)
time2 = time.time()
print('It took %0.3f ms' % ((time2-time1)*1000.0))

It took 35517.060 ms


In [187]:
time1 = time.time()

img_path = "test_img/10.JPEG"
img = image.load_img(img_path, target_size=(224, 224))


x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = x[..., ::-1]

for i in range(3):
    x[:,:,:,i] -= MEAN[i]

x[:,:,:,0] = x[:,:,:,0] * 0.017
x[:,:,:,1] = x[:,:,:,1] * 0.017
x[:,:,:,2] = x[:,:,:,2] * 0.017
            



print('Input image shape:', x.shape)

preds = model2.predict(x)
print(np.argmax(preds, axis=1))
print('Predicted:', decode_predictions(preds))
time2 = time.time()
print('It took %0.3f ms' % ((time2-time1)*1000.0))

Input image shape: (1, 224, 224, 3)
[355]
Predicted: [[('n02437616', 'llama', 0.61203271), ('n01877812', 'wallaby', 0.13150489), ('n01882714', 'koala', 0.044381697), ('n02412080', 'ram', 0.033491764), ('n01518878', 'ostrich', 0.017572388)]]
It took 2109.093 ms
